# Entitlements in TOC-eligible parcels

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import boto3
import utils
import pcts_parser

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

s3 = boto3.client('s3')
bucket_name = 'city-planning-entitlements'

## Parcels
* Figure out how many are duplicates
* Won't know which AINs are used in PCTS, so keep all of them, but have a way to identify how many obs to drop later on

In [3]:
parcels = gpd.read_file(f'zip+s3://{bucket_name}/gis/intermediate/la_parcels_toc.zip')

display(parcels.TOC_Tier.value_counts())
parcels = parcels[parcels.TOC_Tier > 0]

0    442562
1    148994
3    110153
2     65653
4      7844
Name: TOC_Tier, dtype: int64

In [4]:
# Grab the centroids and count number of duplicate obs
parcels2 = utils.get_centroid(parcels)

In [5]:
parcels.drop(columns = 'centroid').to_file(driver = 'GeoJSON', filename = '../gis/intermediate/toc_eligible_parcels.geojson')

## Grab tables from PCTS

In [6]:
cases = catalog.pcts.tCASE.read()
app = catalog.pcts.tAPLC.read()
geo_info = catalog.pcts.tPROP_GEO_INFO.read()
la_prop = catalog.pcts.tLA_PROP.read()

In [7]:
cases1 = cases[['CASE_ID', 'APLC_ID', 'CASE_NBR', 'CASE_SEQ_NBR', 'CASE_YR_NBR', 'CASE_ACTION_ID', 'ADM_ACTION_DT']]
app1 = app[['APLC_ID', 'PROJ_DESC_TXT']]
geo_info1 = geo_info[['CASE_ID', 'PROP_ID']]
la_prop1 = la_prop[la_prop.ASSR_PRCL_NBR.notna()][['PROP_ID', 'ASSR_PRCL_NBR']]

### Subset cases, keep 2016 and after

In [8]:
cases2 = cases1[cases1.CASE_YR_NBR >= 2016]

## Merge with geo_info, la_prop, parcels to ID the parcels that and have entitlements (2016 - after)

In [9]:
m1 = pd.merge(cases2, geo_info1, on = 'CASE_ID', how = 'inner', validate = '1:m')

In [10]:
m2 = pd.merge(m1, la_prop1, on = 'PROP_ID', how = 'inner', validate = 'm:1')

In [11]:
m3 = pd.merge(m2, parcels, left_on = 'ASSR_PRCL_NBR', right_on = 'AIN', how = 'inner', validate = 'm:1')

## Join parcels to zoning and subset to eligible zones
* Subset by eligible zones, see how many TOC-eligible parcels also fall into eligible zones

In [12]:
zoning = gpd.read_file(f's3://{bucket_name}/gis/raw/parsed_zoning.geojson')

In [13]:
eligible_zones = ['R2', 'R3', 'RAS3', 'R4', 'RAS4', 'R5', 
                  'RD1.5', 'RD2', 'RD3', 'RD4', 'RD5', 'RD6', 
                  'C1', 'C2', 'C4', 'C5']

eligible_zoning = zoning[zoning.zone_class.isin(eligible_zones)]

In [14]:
parcels_with_zoning = gpd.sjoin(parcels2, eligible_zoning, how = 'inner', op = 'intersects').drop(columns = ['index_right'])

In [15]:
# Merge in zoning and TOC info about the parcel
m4 = pd.merge(m3, parcels_with_zoning, on = ['AIN', 'centroid', 'TOC_Tier'], how = 'inner')

In [16]:
# Drop those that don't have ADM_ACTION_DT
# Substitute this later with CASE_ACTION_ID
m4 = m4[m4.ADM_ACTION_DT.notna()]

# No more duplicates for same parcel geometry after dropping ADM_ACTION_ID NaTs
display(m4.num_obs.value_counts())

1    10700
Name: num_obs, dtype: int64

In [17]:
# Merge in project description
m5 = pd.merge(m4, app1, on = 'APLC_ID', how = 'inner', validate = 'm:1')

# Drop duplicates
m5 = m5.drop_duplicates()

## Drop duplicates for PCTS entitlements

In [18]:
drop = ['centroid', 'x', 'y', 'obs', 'num_obs', 
        'CASE_ID', 'APLC_ID', 'ASSR_PRCL_NBR', 'PROP_ID', 
        'o1', 'o2', 'o3', 'o1_descrip', 'o2_descrip', 'o3_descrip']

m5 = m5.drop(columns = drop)

In [19]:
# Create new id variable that is just seq number and year. Need a way to get rid of duplicate cases.
m5['id'] = m5.CASE_SEQ_NBR.astype(int).astype(str) + '_' + m5.CASE_YR_NBR.astype(int).astype(str)

In [20]:
cols_we_need = ['id', 'CASE_ACTION_ID', 'ADM_ACTION_DT', 'AIN', 'TOC_Tier', 'ZONE_CMPLT', 'PROJ_DESC_TXT']
m6 = m5.drop_duplicates(subset = cols_we_need)

## Parse the PCTS string and grab suffix

In [21]:
parsed_col_names = ['suffix']

def parse_pcts(row):
    try:
        z = pcts_parser.PCTSCaseNumber(row.CASE_NBR)
        return pd.Series([z.suffix], index = parsed_col_names)
    except ValueError:
        return pd.Series([z.suffix], index = parsed_col_names)

parsed = m6.apply(parse_pcts, axis = 1)

m6 = pd.concat([m6, parsed], axis = 1)

## Additional subsetting for for TOC-eligible parcels

In [22]:
# Tag case as TOC or not
m6['is_TOC'] = m6.CASE_NBR.str.contains('TOC').astype(int)

In [23]:
keep_col = ['CASE_NBR', 'id', 'CASE_ACTION_ID', 'ADM_ACTION_DT', 'AIN', 'TOC_Tier', 'zone_class', 
           'suffix', 'is_TOC']

m6 = m6[keep_col]

## Make into parcel level df

In [24]:
m7 = m6.groupby(['AIN', 'TOC_Tier', 'zone_class', 'is_TOC']).agg({'id':'count'}).reset_index()

In [25]:
# Make wide
m7['num_TOC'] = m7.apply(lambda row: row.id if row.is_TOC == 1 else np.nan, axis = 1) 
m7['num_nonTOC'] = m7.apply(lambda row: row.id if row.is_TOC == 0 else np.nan, axis = 1)

In [26]:
# If there are multiple obs for the same AIN, fill the NaNs with the max from the other column, so we can just drop duplicates after
for col in ['num_TOC', 'num_nonTOC']:
    m7[col] = m7[col].fillna(m7.groupby('AIN')[col].transform('max'))

In [27]:
m8 = m7.drop_duplicates(subset = ['AIN', 'TOC_Tier', 'zone_class', 'num_TOC', 'num_nonTOC'])

for col in ['num_TOC', 'num_nonTOC']:
    m8[col] = m8[col].fillna(0).astype(int)

m8 = m8.drop(columns = ['is_TOC', 'id'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [28]:
toc_parcels = m8[m8.num_TOC > 0]
non_toc_parcels = m8[m8.num_nonTOC > 0]
have_both_parcels = m8[(m8.num_TOC > 0) & (m8.num_nonTOC > 0)]

print(f'# parcels: {len(m8)}')
print(f'# parcels with TOC entitlements: {len(toc_parcels)}')
print(f'# parcels with non TOC entitlements: {len(non_toc_parcels)}')
print(f'# parcels with both TOC and non TOC entitlements: {len(have_both_parcels)}')
print(f'double check sum: {len(toc_parcels) + len(non_toc_parcels) - len(have_both_parcels)}')

# parcels: 2930
# parcels with TOC entitlements: 1156
# parcels with non TOC entitlements: 1845
# parcels with both TOC and non TOC entitlements: 71
double check sum: 2930


In [29]:
print(f'% parcels with TOC entitlements: {len(toc_parcels) / len(m8)}')
print(f'% parcels with non TOC entitlements: {len(non_toc_parcels) / len(m8)}')
print(f'% parcels with both entitlements: {len(have_both_parcels) / len(m8)}')

% parcels with TOC entitlements: 0.3945392491467577
% parcels with non TOC entitlements: 0.6296928327645052
% parcels with both entitlements: 0.024232081911262797


In [30]:
m8.TOC_Tier.value_counts()

3    1421
1     725
2     605
4     179
Name: TOC_Tier, dtype: int64

In [31]:
toc_parcels.zone_class.value_counts()

C2       441
R3       358
R4       191
C4        72
RD1.5     46
RD2       24
R2         7
RAS4       5
RAS3       4
R5         4
C1         4
Name: zone_class, dtype: int64

In [32]:
non_toc_parcels.zone_class.value_counts()

C2       607
R2       330
RD1.5    209
RD2      195
C4       183
R3       160
R4        57
RD3       54
C1        18
C5        17
R5        15
Name: zone_class, dtype: int64

In [33]:
m9 = pd.merge(m8, parcels2, on = ['AIN', 'TOC_Tier'], how = 'inner').drop(columns = ['x', 'y', 'obs', 'num_obs'])

In [34]:
m9.rename(columns = {'centroid':'geometry'}, inplace = True)
m9 = gpd.GeoDataFrame(m9)
m9.crs = {'init':'epsg:2229'}

In [35]:
m9.to_file(driver = 'GeoJSON', filename = '../gis/intermediate/toc_eligible_parcels_with_entitlements.geojson')

## Breakdown by TOC Tiers

In [36]:
m10 = m9.groupby('TOC_Tier').agg({'AIN':'count', 'num_TOC':'sum', 'num_nonTOC':'sum'}).reset_index()

for i in ['TOC', 'nonTOC']:
    new_col = f'pct_{i}'
    numerator = f'num_{i}'
    m10[new_col] = m10[numerator] / (m10.num_TOC + m10.num_nonTOC)
    
m10['all_AIN'] = m10.AIN.sum()
m10['pct_AIN'] = m10.AIN / m10.all_AIN

m10

,TOC_Tier,AIN,num_TOC,num_nonTOC,pct_TOC,pct_nonTOC,all_AIN,pct_AIN
0,1,725,290,607,0.323300,0.676700,2930,0.247440
1,2,605,302,448,0.402667,0.597333,2930,0.206485
2,3,1421,769,1002,0.434218,0.565782,2930,0.484983
3,4,179,75,182,0.291829,0.708171,2930,0.061092


## Breakdown by Zone Class

In [37]:
m11 = m9.groupby('zone_class').agg({'AIN': 'count', 'num_TOC': 'sum', 'num_nonTOC': 'sum'}).reset_index()

for i in ['TOC', 'nonTOC']:
    new_col = f'pct_{i}'
    numerator = f'num_{i}'
    m11[new_col] = m11[numerator] / (m11.num_TOC + m11.num_nonTOC)
    
m11['all_AIN'] = m11.AIN.sum()
m11['pct_AIN'] = m11.AIN / m11.all_AIN

m11

,zone_class,AIN,num_TOC,num_nonTOC,pct_TOC,pct_nonTOC,all_AIN,pct_AIN
0,C1,21,6,19,0.240000,0.760000,2930,0.007167
1,C2,1018,548,769,0.416097,0.583903,2930,0.347440
2,C4,236,93,243,0.276786,0.723214,2930,0.080546
3,C5,17,0,24,0.000000,1.000000,2930,0.005802
4,R2,336,7,409,0.016827,0.983173,2930,0.114676
5,R3,509,449,172,0.723027,0.276973,2930,0.173720
6,R4,237,234,58,0.801370,0.198630,2930,0.080887
7,R5,19,4,21,0.160000,0.840000,2930,0.006485
8,RAS3,4,7,0,1.000000,0.000000,2930,0.001365
9,RAS4,5,6,0,1.000000,0.000000,2930,0.001706


In [38]:
writer = pd.ExcelWriter('../outputs/toc_charts.xlsx', engine='xlsxwriter')

m10.to_excel(writer, sheet_name = 'entitlements_by_tier')
m11.to_excel(writer, sheet_name = 'entitlements_by_zone')

writer.save()